# Prepare a lipids phenotype
In this notebook, we use the UK Biobank phenotype data to prepare a lipids phenotype adjusted for statin use.
  * https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=30690
  * https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=30760
  * https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=30780
  * https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=30870

Note that this work is part of a larger project to [Demonstrate the Potential for Pooled Analysis of All of Us and UK Biobank Genomic Data](https://github.com/all-of-us/ukb-cross-analysis-demo-project). Specifically this is for the portion of the project that is the **siloed** analysis.

# Setup

<div class="alert alert-block alert-warning">
    <b>Cloud Environment</b>: This notebook was written for use on the UK Biobank Research Analysis Platform..
    <ul>
        <li>Use compute type 'Single Node' with sufficient CPU and RAM (e.g. start with 4 CPUs and 15 GB RAM, increase if needed).</li>
        <li>This notebook is pretty fast, but in general it is recommended to be run in the background via <kbd>dx run dxjupyterlab</kbd> to capture provenance.</li>
    </ul>
</div>

```
dx run dxjupyterlab \
    --instance-type=mem2_ssd1_v2_x4 \
    -icmd="papermill 02_ukb_lipids_phenotype.ipynb 02_ukb_lipids_phenotype_$(date +%Y%m%d).ipynb" \
    -iin=02_ukb_lipids_phenotype.ipynb \
    --folder=outputs/r-prepare-phenotype/$(date +%Y%m%d)/
```
See also https://platform.dnanexus.com/app/dxjupyterlab

In [ ]:
lapply(c('lubridate', 'skimr', 'tidyverse'),
       function(pkg) { if(! pkg %in% installed.packages()) { install.packages(pkg)} } )

In [ ]:
library(lubridate)
library(skimr)
library(tidyverse)

In [ ]:
## Plot setup.
theme_set(theme_bw(base_size = 16)) # Default theme for plots.

#' Returns a data frame with a y position and a label, for use annotating ggplot boxplots.
#'
#' @param d A data frame.
#' @return A data frame with column y as max and column label as length.
get_boxplot_fun_data <- function(df) {
  return(data.frame(y = max(df), label = stringr::str_c('N = ', length(df))))
}

## Define constants

In [ ]:
# Papermill parameters. See https://papermill.readthedocs.io/en/latest/usage-parameterize.html

#---[ Inputs ]---
# This was created via ukb_rap_siloed_analyses/01_ukb_lipids_phenotype_retrieval.ipynb
DRUG_MAPPING = '/mnt/project/outputs/spark-pheno-retrieval/20220217/drug_mapping.csv'
# This was created via ukb_rap_siloed_analyses/01_ukb_lipids_phenotype_retrieval.ipynb
PHENO_DATA = '/mnt/project/outputs/spark-pheno-retrieval/20220217/lipids.csv'
# This file was emailed by UK Biobank and manually uploaded to the RAP.
WITHDRAWN_EIDS = '/mnt/project/w7089_20220222.csv'
# Provided by UKB RAP.
EXOME_EIDS = '/mnt/project/Bulk/Exome\\ sequences/Population\\ level\\ exome\\ OQFE\\ variants\\,\\ PLINK\\ format/ukb23155_c21_b0_v1.fam'

#---[ Outputs ]---
PHENOTYPE_FILENAME = 'ukb_200kwes_lipids_phenotype.tsv'

## Retrieve and load the data

In [ ]:
system(str_glue('cp {DRUG_MAPPING} .'), intern=TRUE)
statin_drugs <- read_csv(basename(DRUG_MAPPING))

In [ ]:
system(str_glue('cp {WITHDRAWN_EIDS} .'), intern=TRUE)
withdrawn_eids <- read_tsv(basename(WITHDRAWN_EIDS), col_names = c('eid'))

In [ ]:
system(str_glue('cp {EXOME_EIDS} .'), intern=TRUE)
exome_eids <- read_delim(basename(EXOME_EIDS), delim = ' ', col_names = c('eid'))

In [ ]:
head(exome_eids)

In [ ]:
system(str_glue('cp {PHENO_DATA} .'), intern=TRUE)
raw_pheno <- read_csv(basename(PHENO_DATA))

In [ ]:
spec(raw_pheno)

In [ ]:
#skim(raw_pheno)

# Retain only participants with data in the 200k WES release

In [ ]:
nrow(raw_pheno)

In [ ]:
raw_pheno <- raw_pheno %>%
    filter(eid %in% exome_eids$eid)

In [ ]:
nrow(raw_pheno)

# Subset to our fields of interest 

At this time, we are only interested in the first instance.

In [ ]:
pheno <- raw_pheno %>% select(eid, p31_Sex, contains('_i0_'))

In [ ]:
skim(pheno)

# Omit participants who have withdrawn from the study

In [ ]:
nrow(pheno)

In [ ]:
excluded <- raw_pheno %>%
    filter(eid %in% withdrawn_eids$eid) %>%
    select(eid) %>%
    mutate(exclusion_reason = 'withdrawn from study')

pheno <- pheno %>%
    filter(!eid %in% excluded$eid)

In [ ]:
nrow(pheno)

# Examine the lipids data 

In [ ]:
table(pheno$p30692_i0_Cholesterol_aliquot, useNA = 'always')

In [ ]:
pheno %>%
    mutate(
        missing = is.na(p30690_i0_Cholesterol_mmol_L)
    ) %>%
    group_by(pheno$p30693_i0_Cholesterol_correction_level, missing) %>%
    summarize(count = n())

In [ ]:
pheno %>%
    mutate(
        missing = is.na(p30690_i0_Cholesterol_mmol_L)
    ) %>%
    group_by(p30694_i0_Cholesterol_correction_reason, missing) %>%
    summarize(count = n())

In [ ]:
pheno %>%
    mutate(
        missing = is.na(p30690_i0_Cholesterol_mmol_L)
    ) %>%
    group_by(p30695_i0_Cholesterol_missing_reason, missing) %>%
    summarize(count = n())

## Plot the lipids data 

In [ ]:
options(repr.plot.height = 12, repr.plot.width = 16)

pheno %>%
    filter(!is.na(p21003_i0_Age_when_attended_assessment_centre_years)) %>%
    ggplot(aes(x = cut_width(p21003_i0_Age_when_attended_assessment_centre_years, width = 10, boundary = 0), y = p30690_i0_Cholesterol_mmol_L)) +
    geom_boxplot() +
    stat_summary(fun.data = get_boxplot_fun_data, geom = 'text', size = 4,
                 position = position_dodge(width = 0.9), vjust = -0.8) +
    coord_flip() +
    xlab('age') +
    labs(title = str_glue('Instance 0 measurement per person, by age'),
         caption = 'Source: UK Biobank data')

Pivot the data for easier plotting.

In [ ]:
pheno_check_long <- pheno %>%
    select(eid, p21003_i0_Age_when_attended_assessment_centre_years, ends_with('mmol_L')) %>%
    pivot_longer(
        cols = ends_with('mmol_L'),
        names_to = c('instance', 'measurement'),
        names_pattern = 'p\\d+_(i\\d+)_(.*)_mmol_L',
        values_to = 'mmol_L') %>%
    # Convert to units used by AoU measurements.
    # Formula is from https://www.ncbi.nlm.nih.gov/books/NBK83505/
    mutate(
        mg_dl = case_when(
            measurement == 'HDL_cholesterol' ~ mmol_L * 38.67,
            measurement == 'Cholesterol' ~ mmol_L * 38.67,
            measurement == 'LDL_direct' ~ mmol_L * 38.67,
            measurement == 'Triglycerides' ~ mmol_L * 88.57,
            TRUE ~ NA_real_
        )
    ) %>%
    inner_join(
      pheno %>%
      select(eid, p21003_i0_Age_when_attended_assessment_centre_years, ends_with('correction_reason')) %>%
      pivot_longer(
          cols = ends_with('correction_reason'),
          names_to = c('instance', 'measurement'),
          names_pattern = 'p\\d+_(i\\d+)_(.*)_correction_reason',
          values_to = 'correction_reason')) %>% 
    inner_join(
      pheno %>%
      select(eid, p21003_i0_Age_when_attended_assessment_centre_years, ends_with('missing_reason')) %>%
      pivot_longer(
          cols = ends_with('missing_reason'),
          names_to = c('instance', 'measurement'),
          names_pattern = 'p\\d+_(i\\d+)_(.*)_missing_reason',
          values_to = 'missing_reason')) %>%
    inner_join(
      pheno %>%
      select(eid, p21003_i0_Age_when_attended_assessment_centre_years, ends_with('correction_level')) %>%
      pivot_longer(
          cols = ends_with('correction_level'),
          names_to = c('instance', 'measurement'),
          names_pattern = 'p\\d+_(i\\d+)_(.*)_correction_level',
          values_to = 'correction_level')) %>%
    inner_join(
      pheno %>%
      select(eid, p21003_i0_Age_when_attended_assessment_centre_years, ends_with('aliquot')) %>%
      pivot_longer(
          cols = ends_with('aliquot'),
          names_to = c('instance', 'measurement'),
          names_pattern = 'p\\d+_(i\\d+)_(.*)_aliquot',
          values_to = 'aliquot')) %>%
    inner_join(
      pheno %>%
      select(eid, p21003_i0_Age_when_attended_assessment_centre_years, ends_with('reportability')) %>%
      pivot_longer(
          cols = ends_with('reportability'),
          names_to = c('instance', 'measurement'),
          names_pattern = 'p\\d+_(i\\d+)_(.*)_reportability',
          values_to = 'reportability'))

In [ ]:
# Check the result of the join.
(dim(pheno_check_long))
(nrow(pheno) * 4)
stopifnot(nrow(pheno_check_long) == nrow(pheno) * 4)

In [ ]:
# Uncomment the line below to see row level data.
#head(pheno_check_long)

In [ ]:
pheno_check_long %>%
    filter(!is.na(mg_dl)) %>%
    group_by(missing_reason) %>%
    summarize(count = n())

In [ ]:
pheno_check_long %>%
    group_by(measurement) %>%
    summarize(
        count = n(),
        missing = sum(is.na(mg_dl)),
        median = median(mg_dl, na.rm = TRUE),
        mean = mean(mg_dl, na.rm = TRUE),
        stddev = sd(mg_dl, na.rm = TRUE)
    )

In [ ]:
options(repr.plot.height = 18, repr.plot.width = 16)

pheno_check_long %>%
    filter(!is.na(p21003_i0_Age_when_attended_assessment_centre_years)) %>%
    ggplot(aes(x = cut_width(p21003_i0_Age_when_attended_assessment_centre_years, width = 15, boundary = 0), y = mg_dl)) +
    geom_boxplot() +
    stat_summary(fun.data = get_boxplot_fun_data, geom = 'text', size = 4,
                 position = position_dodge(width = 0.9), vjust = -0.8) +
    scale_y_continuous(breaks = scales::pretty_breaks(n = 10)) +
#    scale_y_log10(breaks = scales::pretty_breaks(n = 10)) +  # Uncomment if the data looks skewed.
    coord_flip() +
    facet_wrap(~ measurement, nrow = length(unique(pheno_check_long$measurement)), scales = 'free_x') +
    xlab('age') +
    labs(title = str_glue('Instance 0 measurement per person, by age'),
         caption = 'Source: UK Biobank data')

# Determine statin use 

In [ ]:
pheno %>%
    select(p20003_i0_Treatment_medication_code) %>%
    filter(str_detect(p20003_i0_Treatment_medication_code, 'statin')) %>%
    head()


In [ ]:
statin_drugs

In [ ]:
(statin_regex = str_c(statin_drugs$drug_name, collapse = '|'))

In [ ]:
str_detect(pheno$p20003_i0_Treatment_medication_code[100], statin_regex)

In [ ]:
pheno <- pheno %>%
    mutate(
        statin_use = case_when(
            str_detect(p20003_i0_Treatment_medication_code, statin_regex) ~ 1,
            TRUE ~ 0
        )
    )

table(pheno$statin_use, useNA = 'always')

# Add the age covariate

In [ ]:
table(pheno$p21003_i0_Age_when_attended_assessment_centre_years, useNA='always')

In [ ]:
nrow(pheno)

In [ ]:
excluded <- bind_rows(
    excluded,
    raw_pheno %>%
        filter(is.na(p21003_i0_Age_when_attended_assessment_centre_years)) %>%
        select(eid) %>%
        mutate(exclusion_reason = 'null age')
    )

pheno <- pheno %>%
    filter(!eid %in% excluded$eid) %>%
    mutate(
        age = p21003_i0_Age_when_attended_assessment_centre_years,
        age2 = age ^2
    )

In [ ]:
nrow(pheno)

In [ ]:
table(pheno$age, useNA='always')

In [ ]:
table(pheno$age2, useNA='always')

# Add the sex covariate

In [ ]:
table(pheno$p31_Sex, useNA='always')

In [ ]:
nrow(pheno)

In [ ]:
excluded <- bind_rows(
    excluded,
    raw_pheno %>%
        filter(is.na(p31_Sex)) %>%
        select(eid) %>%
        mutate(exclusion_reason = 'null sex')
    )

pheno <- pheno %>%
    filter(!eid %in% excluded$eid)

In [ ]:
nrow(pheno)

In [ ]:
table(pheno$p31_Sex, useNA='always')

# Prepare the lipid phenotypes

In [ ]:
nrow(pheno)

Retain only participants with at least one lipid value.

In [ ]:
excluded <- bind_rows(
    excluded,
    raw_pheno %>%
        filter(is.na(p30690_i0_Cholesterol_mmol_L)
               & is.na(p30780_i0_LDL_direct_mmol_L)
               & is.na(p30760_i0_HDL_cholesterol_mmol_L)
               & is.na(p30870_i0_Triglycerides_mmol_L)) %>%
        select(eid) %>%
        mutate(exclusion_reason = 'all null lipids')
    )
    
pheno <- pheno %>%
    filter(!eid %in% excluded$eid)

In [ ]:
nrow(pheno)

## Convert to md/dL

Convert from mmol/L to mg/dL. Formula is from https://www.ncbi.nlm.nih.gov/books/NBK83505/

In [ ]:
pheno <- pheno %>%
    mutate(
        TC_mg_dl = p30690_i0_Cholesterol_mmol_L * 38.67,
        LDL_mg_dl = p30780_i0_LDL_direct_mmol_L * 38.67,
        HDL_mg_dl = p30760_i0_HDL_cholesterol_mmol_L * 38.67,
        TG_mg_dl = p30870_i0_Triglycerides_mmol_L * 88.57
    )

## Adjust for statin use

1.	LDL adjustment based on TG/LDL values 
  1.	`If TG > 400, then LDL = NA`
  2.	`If LDL < 10, then LDL = NA`
2.	LDL and TC adjustment based on Statin (Lipid lowering medication)
  1.	`If STATIN is used, LDL_ADJ = LDL/0.7`
  2.	`If STATIN is used, TOTAL_ADJ = TC/0.8`
3.	TG adjustment
  1.	`TG_LOG = log(TG)`

In [ ]:
pheno <- pheno %>%
    mutate(
        LDL_adj_mg_dl = case_when(
            TG_mg_dl > 400 ~ NA_real_,
            LDL_mg_dl < 10 ~ NA_real_,
            statin_use == 1 ~ LDL_mg_dl / 0.7,
            TRUE ~ LDL_mg_dl
        ),
        TC_adj_mg_dl = ifelse(statin_use == 1, TC_mg_dl / 0.8, TC_mg_dl),
        TG_log_mg_dl = log(TG_mg_dl)
    )

In [ ]:
skim(pheno)

## Plot the adjusted lipids

Pivot the data for easier plotting.

In [ ]:
pheno_long <- pheno %>%
    select(eid, age, age2, p31_Sex, ends_with('mg_dl')) %>%
    pivot_longer(
        cols = ends_with('mg_dl'),
        names_to = c('measurement'),
        names_pattern = '(.*)_mg_dl',
        values_to = 'mg_dl')

In [ ]:
pheno_long %>%
    group_by(measurement) %>%
    summarize(
        count = n(),
        missing = sum(is.na(mg_dl)),
        median = median(mg_dl, na.rm = TRUE),
        mean = mean(mg_dl, na.rm = TRUE),
        stddev = sd(mg_dl, na.rm = TRUE)
    )

In [ ]:
options(repr.plot.height = 24, repr.plot.width = 16)

pheno_long %>%
    ggplot(aes(x = cut_width(age, width = 15, boundary = 0), y = mg_dl)) +
    geom_boxplot() +
    stat_summary(fun.data = get_boxplot_fun_data, geom = 'text', size = 4,
                 position = position_dodge(width = 0.9), vjust = -0.8) +
    scale_y_continuous(breaks = scales::pretty_breaks(n = 10)) +
#    scale_y_log10(breaks = scales::pretty_breaks(n = 10)) +  # Uncomment if the data looks skewed.
    coord_flip() +
    facet_wrap(~ measurement, nrow = length(unique(pheno_long$measurement)), scales = 'free_x') +
    xlab('age') +
    labs(title = str_glue('Instance 0 measurement per person, by age'),
         caption = 'Source: UK Biobank data')

# Check our exclusions

In [ ]:
excluded %>%
    group_by(eid) %>%
    mutate(
        exclusion_reasons = str_c(exclusion_reason, collapse = ', ')
    ) %>%
    group_by(exclusion_reasons) %>%
    summarize(
        count = n()
    )

In [ ]:
stopifnot(nrow(pheno) == nrow(raw_pheno) - length(unique(excluded$eid)))

# Retain only needed columns

Also rename and reorder them per regenie's requirements.

In [ ]:
pheno <- pheno %>%
    select(FID=eid, IID=eid, sex=p31_Sex, age, age2, statin_use, TC_mg_dl, LDL_mg_dl, HDL_mg_dl,
           TG_mg_dl, LDL_adj_mg_dl, TC_adj_mg_dl, TG_log_mg_dl)

In [ ]:
skim(pheno)

# Write the prepared phenotype to a file

In [ ]:
write_tsv(pheno, PHENOTYPE_FILENAME)

# Provenance 

In [ ]:
devtools::session_info()